<a href="https://colab.research.google.com/github/Sakib635/sage2.0/blob/main/5th_copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install z3-solver


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 MB 13.5 MB/s eta 0:00:00


In [2]:
import os
import json
import re
import time
import logging
from z3 import Solver, Bool, Or, And, Implies, sat, Int, String, Not, Real, simplify, Optimize

In [18]:
def parse_requirements(requirements_txt):
    """
    Parses the content of requirements.txt into a dictionary of packages and their version specifiers.
    Handles cases where version specifiers might include an "extra" part after a semicolon, and cases where there are no version specifiers.

    Parameters:
        requirements_txt (str): The content of the requirements.txt file as a single string.

    Returns:
        dict: A dictionary where keys are package names and values are lists of tuples representing version specifiers.
              If a package has no version specifiers, the value will be an empty list.
    """
    requirements = {}  # Initialize an empty dictionary to store package requirements
    lines = requirements_txt.strip().split('\n')  # Split the input string into lines
    for line in lines:  # Iterate through each line
        line = line.strip()  # Trim whitespace from the line
        if line:  # Check if the line is not empty
            # Split the line into parts based on version specifiers using a regular expression
            parts = re.split(r'([><!=]=?[\d.*]+(?:, )?)', line)
            # Extract the package name from the first part, handling extra parts after a semicolon
            package = parts[0].strip().split(';')[0].strip()
            version_specs = parts[1:]  # Extract version specifiers from the remaining parts

            if not version_specs:  # If there are no version specifiers
                requirements[package] = []  # Add the package with an empty list of specifiers
            else:
                for spec in version_specs:  # Iterate through each version specifier
                    if spec.strip():  # Trim whitespace from the specifier
                        # Match the specifier with a regular expression to extract the operator and version
                        match = re.match(r'([><!=]=?)([\d.*]+)', spec.strip())
                        if match:  # If the match is successful
                            operator, version = match.groups()  # Extract the operator and version
                            if package in requirements:  # If the package is already in the dictionary
                                requirements[package].append((operator, version))  # Append the (operator, version) tuple to the list
                            else:
                                requirements[package] = [(operator, version)]  # Add the package with a new list containing the (operator, version) tuple
    return requirements  # Return the dictionary of requirements


In [19]:
def version_satisfies(version, spec):
    """
    Check if a version satisfies a given version specifier.

    Args:
    version (str): The version of the package.
    spec (tuple): A tuple containing the version operator and version number.

    Returns:
    bool: True if the version satisfies the specifier, False otherwise.
    """
    operator, spec_version = spec
    if operator == '==':
        return re.match(spec_version.replace('*', '.*'), version) is not None
    elif operator == '!=':
        return re.match(spec_version.replace('*', '.*'), version) is None
    elif operator == '>':
        return version > spec_version
    elif operator == '>=':
        return version >= spec_version
    elif operator == '<':
        return version < spec_version
    elif operator == '<=':
        return version <= spec_version
    return False

def find_matching_versions(package, specs, projects_data):
    """
    Find all versions of a package that satisfy the given version specifiers.

    Args:
    package (str): The name of the package.
    specs (list): A list of tuples containing version operators and version numbers.
    projects_data (dict): A dictionary containing project data with available versions.

    Returns:
    list: A list of versions that satisfy the given specs.
    """
    if package not in projects_data:
        return []

    versions = projects_data[package].keys()  # Get all versions of the package.
    matching_versions = []
    for version in versions:
        if all(version_satisfies(version, spec) for spec in specs):
            matching_versions.append(version)  # Add version if it satisfies all specs.
    return matching_versions

def fetch_direct_dependencies(requirements, projects_data):
    """
    Fetch direct dependencies for each package based on the parsed requirements.

    Args:
    requirements (dict): A dictionary where keys are package names and values are lists of tuples with version specs.
    projects_data (dict): A dictionary containing project data with available versions and dependencies.

    Returns:
    dict: A dictionary where keys are package names and values are lists of matching versions.
    """
    direct_dependencies = {}

    for package, specs in requirements.items():
        matching_versions = find_matching_versions(package, specs, projects_data["projects"])
        direct_dependencies[package] = matching_versions  # Store matching versions for the package.

    return direct_dependencies


In [20]:
def parse_dependency(dependency):
    """
    Parse a dependency string into a package and a list of version specifiers.

    Parameters:
        dependency (str): A string representing a package and its version specifiers.

    Returns:
        tuple: A tuple where the first element is the package name and the second element is a list of tuples representing version specifiers.
    """
    # Split the dependency string into parts based on version specifiers using a regular expression
    parts = re.split(r'([><!=]=?[\d.*]+(?:, )?)', dependency)
    # Extract the package name from the first part, handling extra parts after a semicolon
    package = parts[0].strip().split(';')[0].strip()
    version_specs = []  # Initialize an empty list to store version specifiers

    # Iterate through the remaining parts
    for spec in parts[1:]:
        if spec.strip():  # Trim whitespace from the specifier
            # Match the specifier with a regular expression to extract the operator and version
            match = re.match(r'([><!=]=?)([\d.*]+)', spec.strip())
            if match:  # If the match is successful
                operator, version = match.groups()  # Extract the operator and version
                version_specs.append((operator, version))  # Append the (operator, version) tuple to version_specs

    return package, version_specs  # Return the package and version_specs

def fetch_transitive_dependencies(direct_dependencies, projects_data):
    """
    Recursively fetch transitive dependencies for each version of the packages in direct dependencies.

    Parameters:
        direct_dependencies (dict): A dictionary of direct dependencies where keys are package names and values are lists of versions.
        projects_data (dict): A dictionary containing project data, including available versions and their dependencies.

    Returns:
        dict: A dictionary where keys are package versions and values are dictionaries of transitive dependencies.
    """
    transitive_dependencies = {}  # Initialize an empty dictionary to store transitive dependencies

    def _fetch(package, version):
        key = f"{package}=={version}"  # Create a key as "package==version"
        if key in transitive_dependencies:  # If the key is already in transitive_dependencies, return the stored value
            return transitive_dependencies[key]

        # Handle case sensitivity for package lookup
        version_data = projects_data["projects"].get(package, {}).get(version, {})
        if not version_data:  # If version_data is empty, try lowercase version of the package name
            version_data = projects_data["projects"].get(package.lower(), {}).get(version, {})

        dependencies = {}  # Initialize an empty dictionary to store dependencies
        if version_data.get("dependency_packages"):  # If version_data contains dependency_packages
            for dep in version_data["dependency_packages"]:
                dep_package, dep_specs = parse_dependency(dep)  # Parse dep to get dep_package and dep_specs

                # Handle case sensitivity for dependency package lookup
                matching_versions = []
                if not dep_specs:  # If no version specifiers are provided, fetch all versions of the dependency package
                    matching_versions = list(projects_data["projects"].get(dep_package, {}).keys())
                    if not matching_versions:  # Try lowercase version of the dependency package name if no versions found
                        matching_versions = list(projects_data["projects"].get(dep_package.lower(), {}).keys())
                else:  # If there are version specifiers, fetch matching versions of the dependency package
                    matching_versions = find_matching_versions(dep_package, dep_specs, projects_data["projects"])

                dependencies[dep_package] = matching_versions  # Add dep_package with matching versions to dependencies
                for dep_version in matching_versions:  # Recursively fetch dependencies for each matching version
                    _fetch(dep_package, dep_version)

        # Ensure that an empty dictionary is assigned if no dependencies are found
        transitive_dependencies[key] = dependencies
        return dependencies  # Return the dependencies for this package and version

    for package, versions in direct_dependencies.items():  # Iterate through direct dependencies
        for version in versions:  # Iterate through each version of the package
            _fetch(package, version)  # Fetch transitive dependencies for the package and version

    return transitive_dependencies  # Return the transitive dependencies dictionary


In [21]:
def generate_smt_expression(direct_dependencies, transitive_dependencies):
    """
    Generate an SMT (Satisfiability Modulo Theories) expression to handle package version constraints,
    including both direct and transitive dependencies, using an Optimize solver.

    Args:
    direct_dependencies (dict): A dictionary where keys are package names and values are lists of matching versions.
    transitive_dependencies (dict): A dictionary where keys are "package==version" and values are dictionaries of transitive dependencies.

    Returns:
    tuple: An Optimize solver instance with the added constraints and the list of constraints.
    """
    # Initialize an Optimize solver to handle both hard and soft constraints
    solver = Optimize()
    constraints = []

    # Generate constraints for direct dependencies
    for package, versions in direct_dependencies.items():
        if isinstance(versions, list):
            # Create a constraint that the package version must be one of the specified versions
            package_constraint = Or([String(package) == v for v in versions])
            constraints.append(package_constraint)
            # Add soft constraints with weights for versions
            sorted_versions = sorted(versions, reverse=False)  # Sort versions to prioritize newer versions
            weight = 1
            for version in sorted_versions:
                # Add a soft constraint with increasing weight for newer versions
                solver.add_soft(String(package) == version, weight)
                weight += 1  # Increment the weight for the next version

    # Generate constraints for transitive dependencies
    for package_version, dependencies in transitive_dependencies.items():
        if isinstance(dependencies, dict):
            # Split the package_version to get the package name and its version
            package, version = package_version.split('==')
            for dep_package, dep_versions in dependencies.items():
                # Create a constraint for each dependency that it must be one of the specified versions
                dependency_constraint = Or([String(dep_package) == dep_version for dep_version in dep_versions])
                constraints.append(Implies(String(package) == version, dependency_constraint))
                # Add soft constraints with weights for versions
                sorted_versions = sorted(dep_versions, reverse=False)  # Sort versions to prioritize newer versions
                weight = 1
                for dep_version in sorted_versions:
                    # Add a soft constraint with increasing weight for newer versions
                    solver.add_soft(String(dep_package) == dep_version, weight)
                    weight += 1  # Increment the weight for the next version

    # Combine all constraints into a single final constraint
    final_constraint = And(constraints)
    solver.add(final_constraint)

    return solver, constraints


In [49]:

def smt_solver(solver):
    """
    Solve the SMT (Satisfiability Modulo Theories) expression using the provided solver.

    Args:
    solver (Optimize): An Optimize solver instance with added constraints.

    Returns:
    dict or None: A dictionary representing the solution model if satisfiable, otherwise None.
    """
    start_time = time.time()
    # Check for the maximum satisfaction
    if solver.check() == sat:
        # Get the model
        model = solver.model()
        elapsed_time = time.time() # Calculate elapsed time before returning
        # Return the solution model as a dictionary
        return {d.name(): model[d] for d in model.decls()}, start_time, elapsed_time
    else:
        # Print a message if the constraints are not satisfiable
        print("Not satisfiable.")
        return None, None, None # Return None for all values if no solution

In [223]:
def read_requirements(directory):
    """
    Reads the content of the `requirements.txt` file from a specified directory.

    Parameters:
        directory (str): The path to the directory containing the `requirements.txt` file.

    Returns:
        str: The content of the `requirements.txt` file as a single string.
    """
    with open(os.path.join(directory, 'r30.txt'), 'r') as file:
        return file.read()

# Function to read the JSON file from a directory
def read_json_file(directory, filename='updated_formated_8k.json'):
    """
    Reads the content of a JSON file from a specified directory.

    Parameters:
        directory (str): The path to the directory containing the JSON file.
        filename (str): The name of the JSON file to read. Default is 'updated_formated_8k.json'.

    Returns:
        dict: The content of the JSON file as a dictionary.
    """
    with open(os.path.join(directory, filename), 'r') as file:
        return json.load(file)

In [224]:
import time

def main():
    """
    Main function to execute the dependency resolution process, including reading files,
    parsing requirements, fetching dependencies, generating SMT expressions, and solving them.
    """
    directory = '/content/drive/MyDrive/smart pip sample data'

    # Log file setup
    log_file = 'execution_log.txt'

    def log_execution_time(action_name, start_time, end_time):
        """
        Log the execution time of a specific action to a log file.

        Args:
        action_name (str): The name of the action being logged.
        start_time (float): The start time of the action.
        end_time (float): The end time of the action.
        """
        with open(log_file, 'a') as file:
            file.write(f'{action_name} execution time: {end_time - start_time} seconds\n')

    # Read files from the directory
    start_time = time.time()
    requirements_txt = read_requirements(directory)
    projects_data = read_json_file(directory)
    end_time = time.time()
    log_execution_time("Reading files", start_time, end_time)

    # Parse requirements
    start_time = time.time()
    requirements = parse_requirements(requirements_txt)
    # assert parse_requirements("python-sat>=3.1") == ("python-sat", [(">=", 3.1)])
    end_time = time.time()
    log_execution_time("Parsing requirements", start_time, end_time)
    print("Parsed requirements:", requirements)

    # Fetch matching versions and their dependencies
    start_time = time.time()
    direct_dependencies = fetch_direct_dependencies(requirements, projects_data)
    end_time = time.time()
    log_execution_time("Fetching versions and dependencies", start_time, end_time)
    print("Direct dependencies:", direct_dependencies)

    # Fetch transitive dependencies
    start_time = time.time()
    transitive_dependencies = fetch_transitive_dependencies(direct_dependencies, projects_data)
    end_time = time.time()
    log_execution_time("Fetching transitive dependencies", start_time, end_time)
    print("Transitive dependencies:", transitive_dependencies)

    # Generate SMT expression
    start_time = time.time()
    solver, constraints = generate_smt_expression(direct_dependencies, transitive_dependencies)
    end_time = time.time()
    log_execution_time("Generating SMT expression", start_time, end_time)

    # Save SMT expression to a file (optional)
    with open('SMT_expression.txt', 'w') as file:
        file.write(str(solver))

    # Solve the SMT expression

    solution, start_time, end_time = smt_solver(solver)

    if solution: # Check if a solution was found before logging and printing
        log_execution_time("Solving SMT expression", start_time, end_time)
        print(f'Optimal Solution: {solution}')
if __name__ == "__main__":
    main()


Parsed requirements: {'Django': [('==', '3.1.7')], 'utils': [('==', '1.0.1')]}
Direct dependencies: {'Django': ['3.1.7'], 'utils': ['1.0.1']}
Transitive dependencies: {'asgiref==3.8.1': {}, 'asgiref==3.8.0': {}, 'asgiref==3.7.2': {}, 'asgiref==3.7.1': {}, 'asgiref==3.7.0': {}, 'asgiref==3.6.0': {}, 'asgiref==3.5.2': {}, 'asgiref==3.5.1': {}, 'asgiref==3.5.0': {}, 'asgiref==3.4.1': {}, 'asgiref==3.4.0': {}, 'asgiref==3.3.4': {}, 'asgiref==3.3.3': {}, 'asgiref==3.3.2': {}, 'asgiref==3.3.1': {}, 'asgiref==3.3.0': {}, 'asgiref==3.2.10': {}, 'asgiref==3.2.9': {}, 'asgiref==3.2.8': {}, 'asgiref==3.2.7': {}, 'asgiref==3.2.6': {}, 'asgiref==3.2.5': {}, 'asgiref==3.2.4': {}, 'asgiref==3.2.3': {}, 'asgiref==3.2.2': {}, 'pytz==2004a': {}, 'pytz==2004b': {}, 'pytz==2004b.2': {}, 'pytz==2004d': {}, 'pytz==2005a': {}, 'pytz==2005e': {}, 'pytz==2005i': {}, 'pytz==2005k': {}, 'pytz==2005m': {}, 'pytz==2005r': {}, 'pytz==2006g': {}, 'pytz==2006j': {}, 'pytz==2006p': {}, 'pytz==2007c': {}, 'pytz==2007d'